In [1]:
import pandas as pd
import numpy as np
from workalendar.usa import UnitedStates

cal = UnitedStates()

# 1. Read VBA sheet as main data framework
print("Reading ARKK VBA data...")
complete_data = pd.read_excel('ARKK521.xlsx', sheet_name='VBA')
complete_data['Date'] = pd.to_datetime(complete_data['Date'])

# 2. Extract company names and industry information for lookup
print("Getting company names and industry info...")
stocks_info = pd.read_excel('arkstocksinfo.xlsx', sheet_name='Sheet1')

# Use Name (B column) to match and get H column (long_comp_name) content
# B column = "Bloomberg Name", H column = "long_comp_name"
name_lookup = stocks_info.drop_duplicates('Bloomberg Name').set_index('Bloomberg Name')['long_comp_name'].to_dict()
industry_lookup = stocks_info.drop_duplicates('Bloomberg Name').set_index('Bloomberg Name')['gics_industry_name'].to_dict()

# Add columns by mapping Name to company info
complete_data['Company_Name'] = complete_data['Name'].map(name_lookup)
complete_data['Industry'] = complete_data['Name'].map(industry_lookup)

print(f"Company names found for {complete_data['Company_Name'].notna().sum():,} records")
print(f"Industry info found for {complete_data['Industry'].notna().sum():,} records")

# 3. Process price data
print("Processing price data...")
df_raw = pd.read_excel('ALLARK521StocksNoDiv.xlsx', sheet_name='Sheet1', header=None)

# Extract Bloomberg Name and prices
price_lookup = {}
for i in range(2, df_raw.shape[1], 2):
    # Check if both date and price columns exist
    if i < df_raw.shape[1] and (i + 1) < df_raw.shape[1]:
        bloomberg_name = df_raw.iloc[1, i]  # Row 1: Bloomberg Name
        if pd.notna(bloomberg_name):
            stock_prices = {}
            date_col = i
            price_col = i + 1
            
            for row in range(2, df_raw.shape[0]):
                # Add safety check for row access
                if row < df_raw.shape[0]:
                    date_val = df_raw.iloc[row, date_col]
                    price_val = df_raw.iloc[row, price_col]
                    
                    if pd.notna(date_val) and pd.notna(price_val):
                        try:
                            date_obj = pd.to_datetime(date_val)
                            stock_prices[date_obj] = float(price_val)
                        except:
                            continue
            
            if stock_prices:
                price_lookup[bloomberg_name] = stock_prices

print(f"Found price data for {len(price_lookup)} stocks")

# 4. Add stock price column
complete_data['Stock_Price'] = complete_data.apply(
    lambda row: price_lookup.get(row['Name'], {}).get(row['Date'], np.nan), axis=1
)

# 5. Filter business days only
print("Filtering business days...")
original_count = len(complete_data)
complete_data = complete_data[complete_data['Date'].apply(lambda x: cal.is_working_day(x.date()))]
print(f"Filtered: {original_count:,} -> {len(complete_data):,} rows")

# 6. Set date as index
complete_data.set_index('Date', inplace=True)
complete_data.sort_index(inplace=True)

# 7. Display final results
print(f"\n" + "="*60)
print("📊 FINAL DATAFRAME")
print("="*60)
print(f"Shape: {complete_data.shape[0]:,} rows × {complete_data.shape[1]} columns")
print(f"Date range: {complete_data.index.min().strftime('%Y-%m-%d')} to {complete_data.index.max().strftime('%Y-%m-%d')}")

print(f"\nColumns:")
for i, col in enumerate(complete_data.columns, 1):
    non_null = complete_data[col].notna().sum()
    print(f"{i:2d}. {col:<20} ({non_null:,} non-null)")

print(f"\nSample data:")
print(complete_data[['Company_Name', 'Industry', 'Stock_Price']].head())

# Statistics
price_missing = complete_data['Stock_Price'].isnull().sum()
print(f"\nStock_Price missing: {price_missing:,} / {len(complete_data):,} ({price_missing/len(complete_data)*100:.1f}%)")



Reading ARKK VBA data...
Getting company names and industry info...
Company names found for 170,323 records
Industry info found for 161,049 records
Processing price data...


/Users/mfr/Library/CloudStorage/Dropbox/Robyn/Python_Projects/Finance_environment/finance_venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell RZ3 is marked as a date but the serial value 7916800000 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


Found price data for 434 stocks
Filtering business days...
Filtered: 170,323 -> 116,862 rows

📊 FINAL DATAFRAME
Shape: 116,862 rows × 11 columns
Date range: 2014-10-31 to 2025-05-21

Columns:
 1. Name                 (116,862 non-null)
 2. Weight               (116,862 non-null)
 3. Position             (116,862 non-null)
 4. Market Value         (116,862 non-null)
 5. CUSIP                (104,900 non-null)
 6. BBID                 (116,862 non-null)
 7. ISIN                 (113,749 non-null)
 8. Fund Flows           (116,862 non-null)
 9. Company_Name         (116,862 non-null)
10. Industry             (110,478 non-null)
11. Stock_Price          (114,059 non-null)

Sample data:
                Company_Name                                  Industry  \
Date                                                                     
2014-10-31  MercadoLibre Inc                          Broadline Retail   
2014-10-31      Autodesk Inc                                  Software   
2014-10-31    

In [2]:
# Complete ARK Data Processing Pipeline
# 1. Remove low quality dates → 2. Clean data & calculate weights

import numpy as np

# ============================================================================
# STEP 1: ANALYZE AND REMOVE DATES WITH POOR PRICE COVERAGE
# ============================================================================

print(f"📊 STEP 1: ANALYZING PRICE COVERAGE BY DATE")
print("="*60)

# Calculate price coverage by date
daily_stats = complete_data.groupby(complete_data.index).agg({
    'Stock_Price': lambda x: x.notna().sum(),
    'Name': 'count'
})
daily_stats.columns = ['Tickers_With_Price', 'Total_Tickers']
daily_stats['Price_Coverage_Pct'] = (daily_stats['Tickers_With_Price'] / daily_stats['Total_Tickers'] * 100).round(1)

print(f"Overall price coverage: {daily_stats['Price_Coverage_Pct'].mean():.1f}%")

# Show worst dates
worst_dates = daily_stats.nsmallest(10, 'Price_Coverage_Pct')
print(f"\nWorst 10 dates (price coverage):")
for date, row in worst_dates.iterrows():
    print(f"   {date.strftime('%Y-%m-%d')}: {row['Tickers_With_Price']:.0f}/{row['Total_Tickers']:.0f} ({row['Price_Coverage_Pct']:.1f}%)")

# Remove dates with <50% coverage
threshold = 50.0
dates_to_remove = daily_stats[daily_stats['Price_Coverage_Pct'] < threshold].index

if len(dates_to_remove) > 0:
    print(f"\nRemoving {len(dates_to_remove)} dates with <{threshold}% price coverage...")
    original_count = len(complete_data)
    complete_data = complete_data[~complete_data.index.isin(dates_to_remove)]
    removed_count = original_count - len(complete_data)
    print(f"✅ Removed {removed_count:,} records from {len(dates_to_remove):,} dates")
else:
    print(f"✅ No dates with <{threshold}% coverage found")

# ============================================================================
# STEP 2: DATA CLEANING AND WEIGHT CALCULATION
# ============================================================================


# Remove USD entries
original_count = len(complete_data)
complete_data = complete_data[complete_data['Name'] != 'USD Curncy']
usd_removed = original_count - len(complete_data)

# Recalculate Market Value
complete_data['Market Value'] = np.nan
mask = (complete_data['Position'].notna()) & (complete_data['Stock_Price'].notna())
complete_data.loc[mask, 'Market Value'] = complete_data.loc[mask, 'Position'] * complete_data.loc[mask, 'Stock_Price']
calculated_count = complete_data['Market Value'].notna().sum()

# Calculate ETF Market Value and Weights
complete_data['Weight'] = np.nan

# Daily ETF market values
daily_total_mv = complete_data.groupby(complete_data.index)['Market Value'].sum()
complete_data['ETF Market Value'] = complete_data.index.map(daily_total_mv)

# Calculate weights
weight_mask = ((complete_data['Market Value'].notna()) & 
              (complete_data['ETF Market Value'].notna()) & 
              (complete_data['ETF Market Value'] != 0))
complete_data.loc[weight_mask, 'Weight'] = (
    complete_data.loc[weight_mask, 'Market Value'] / 
    complete_data.loc[weight_mask, 'ETF Market Value']
)

weight_calculated = complete_data['Weight'].notna().sum()

# Validate weights
weight_sums = complete_data.groupby(complete_data.index)['Weight'].sum()
tolerance = 0.001
problematic_dates = weight_sums[abs(weight_sums - 1.0) > tolerance]

if len(problematic_dates) > 0:
    print(f"⚠️  {len(problematic_dates)} dates with weight sum ≠ 1")
else:
    print("✅ All dates have weight sum ≈ 1.0")

# ============================================================================
# STEP 3: FINAL EXPORT TO EXCEL
# ============================================================================

# Prepare for export
export_data = complete_data.reset_index()
export_data['Date'] = export_data['Date'].dt.date

# Remove unwanted columns
columns_to_remove = ['CUSIP', 'ISIN', 'BBID']
for col in columns_to_remove:
    if col in export_data.columns:
        export_data = export_data.drop(columns=[col])

# Reorder columns
column_order = ['Date', 'Name', 'Company_Name', 'Industry', 'Position', 'Stock_Price', 
                'Market Value', 'ETF Market Value', 'Weight', 'Fund']

existing_cols = [col for col in column_order if col in export_data.columns]
other_cols = [col for col in export_data.columns if col not in column_order]
final_column_order = existing_cols + other_cols
export_data = export_data[final_column_order]

# Export to Excel
filename = 'ark_data_complete_processed.xlsx'
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    export_data.to_excel(writer, sheet_name='ARK_Data_Final', index=False)
    
    # Auto-adjust column widths
    workbook = writer.book
    worksheet = writer.sheets['ARK_Data_Final']
    
    for column in worksheet.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 50)
        worksheet.column_dimensions[column_letter].width = adjusted_width

# ============================================================================
# STEP 4: DATA QUALITY CHECKS
# ============================================================================

print(f"\nDATA QUALITY CHECKS")
print("="*60)

# Check 1: Days with identical stock prices for multiple stocks (>5 stocks)
daily_price_duplicates = complete_data.groupby(complete_data.index).apply(
    lambda x: (x['Stock_Price'].dropna().value_counts() > 5).any() if len(x['Stock_Price'].dropna()) > 5 else False
)
days_with_duplicate_prices = daily_price_duplicates.sum()
total_days = len(daily_price_duplicates)
print(f"1. Days with >5 stocks having same price: {days_with_duplicate_prices}/{total_days} ({days_with_duplicate_prices/total_days*100:.1f}%)")

if days_with_duplicate_prices > 0:
    duplicate_price_dates = daily_price_duplicates[daily_price_duplicates].index
    print(f"   Problem dates: {', '.join(duplicate_price_dates[:5].strftime('%Y-%m-%d'))}" + 
          (f" (+{len(duplicate_price_dates)-5} more)" if len(duplicate_price_dates) > 5 else ""))

# Check 2: Days with identical positions for multiple stocks (>5 stocks)
daily_position_duplicates = complete_data.groupby(complete_data.index).apply(
    lambda x: (x['Position'].dropna().value_counts() > 5).any() if len(x['Position'].dropna()) > 5 else False
)
days_with_duplicate_positions = daily_position_duplicates.sum()
print(f"\n2. Days with >5 stocks having same position: {days_with_duplicate_positions}/{total_days} ({days_with_duplicate_positions/total_days*100:.1f}%)")

if days_with_duplicate_positions > 0:
    duplicate_position_dates = daily_position_duplicates[daily_position_duplicates].index
    print(f"   Problem dates: {', '.join(duplicate_position_dates[:5].strftime('%Y-%m-%d'))}" + 
          (f" (+{len(duplicate_position_dates)-5} more)" if len(duplicate_position_dates) > 5 else ""))

# Check 3: Days with >90% stocks having no position changes from previous day
def check_position_changes():
    # Calculate position changes for each stock
    complete_data_sorted = complete_data.sort_index()
    complete_data_sorted['Position_Prev'] = complete_data_sorted.groupby('Name')['Position'].shift(1)
    complete_data_sorted['Position_Changed'] = (
        complete_data_sorted['Position'] != complete_data_sorted['Position_Prev']
    ) | complete_data_sorted['Position_Prev'].isna()
    
    # Calculate daily percentage of stocks with no position changes
    daily_no_change_pct = complete_data_sorted.groupby(complete_data_sorted.index).apply(
        lambda x: (~x['Position_Changed']).sum() / len(x) * 100 if len(x) > 0 else 0
    )
    
    return daily_no_change_pct > 90

daily_position_no_change = check_position_changes()
days_no_position_change = daily_position_no_change.sum()
print(f"\n3. Days with >90% stocks unchanged positions: {days_no_position_change}/{total_days} ({days_no_position_change/total_days*100:.1f}%)")

if days_no_position_change > 0:
    no_change_dates = daily_position_no_change[daily_position_no_change].index
    print(f"   Problem dates: {', '.join(no_change_dates[:5].strftime('%Y-%m-%d'))}" + 
          (f" (+{len(no_change_dates)-5} more)" if len(no_change_dates) > 5 else ""))

# Check 4: Days missing stock price information (excluding 100% missing tickers)
# First identify tickers with 100% missing prices
ticker_price_coverage = complete_data.groupby('Name')['Stock_Price'].apply(
    lambda x: x.notna().sum() / len(x) * 100
)
tickers_with_some_prices = ticker_price_coverage[ticker_price_coverage > 0].index

# Only count missing prices for tickers that have some price data
filtered_data = complete_data[complete_data['Name'].isin(tickers_with_some_prices)]
daily_missing_prices = filtered_data.groupby(filtered_data.index)['Stock_Price'].apply(
    lambda x: x.isna().sum()
)
days_with_missing_prices = (daily_missing_prices > 0).sum()
print(f"\n4. Days with missing prices (excluding 100% missing tickers): {days_with_missing_prices}/{total_days} ({days_with_missing_prices/total_days*100:.1f}%)")

if days_with_missing_prices > 0:
    worst_missing_days = daily_missing_prices[daily_missing_prices > 0].sort_values(ascending=False)
    print(f"   Worst dates: {', '.join(worst_missing_days.head(5).index.strftime('%Y-%m-%d'))}" + 
          (f" (+{len(worst_missing_days)-5} more)" if len(worst_missing_days) > 5 else ""))

# Check 5: Tickers missing stock price information
ticker_missing_prices = complete_data.groupby('Name')['Stock_Price'].apply(
    lambda x: x.isna().sum() / len(x) * 100
)
tickers_with_missing = ticker_missing_prices[ticker_missing_prices > 0].sort_values(ascending=False)
print(f"\n5. Tickers with missing prices: {len(tickers_with_missing)}/{len(ticker_missing_prices)} ({len(tickers_with_missing)/len(ticker_missing_prices)*100:.1f}%)")

if len(tickers_with_missing) > 0:
    print(f"   Worst tickers:")
    for ticker, missing_pct in tickers_with_missing.head(20).items():
        print(f"     {ticker}: {missing_pct:.1f}% missing")


📊 STEP 1: ANALYZING PRICE COVERAGE BY DATE
Overall price coverage: 97.5%

Worst 10 dates (price coverage):
   2015-04-03: 1/57 (1.8%)
   2021-04-02: 1/57 (1.8%)
   2018-03-30: 1/53 (1.9%)
   2017-04-14: 1/50 (2.0%)
   2016-03-25: 1/43 (2.3%)
   2018-12-05: 1/39 (2.6%)
   2019-04-19: 1/39 (2.6%)
   2024-03-29: 1/39 (2.6%)
   2022-04-15: 1/37 (2.7%)
   2025-04-18: 1/37 (2.7%)

Removing 16 dates with <50.0% price coverage...
✅ Removed 656 records from 16 dates
⚠️  1 dates with weight sum ≠ 1

DATA QUALITY CHECKS
1. Days with >5 stocks having same price: 0/2632 (0.0%)

2. Days with >5 stocks having same position: 3/2632 (0.1%)
   Problem dates: 2015-02-06, 2020-03-19, 2021-03-25

3. Days with >90% stocks unchanged positions: 692/2632 (26.3%)
   Problem dates: 2014-11-03, 2014-11-04, 2014-11-05, 2014-11-06, 2014-11-10 (+687 more)

4. Days with missing prices (excluding 100% missing tickers): 45/2632 (1.7%)
   Worst dates: 2015-01-02, 2015-01-16, 2015-01-23, 2015-01-30, 2015-02-06 (+40 more)

In [3]:

print("DataFrame Info:")
complete_data.info()

print("\nFirst 5 rows:")
complete_data.head()

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 113650 entries, 2014-10-31 to 2025-05-21
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Name              113650 non-null  object 
 1   Weight            111427 non-null  float64
 2   Position          113650 non-null  float64
 3   Market Value      111483 non-null  float64
 4   CUSIP             104313 non-null  object 
 5   BBID              113650 non-null  object 
 6   ISIN              113115 non-null  object 
 7   Fund Flows        113650 non-null  float64
 8   Company_Name      113650 non-null  object 
 9   Industry          109860 non-null  object 
 10  Stock_Price       111483 non-null  float64
 11  ETF Market Value  113650 non-null  float64
dtypes: float64(6), object(6)
memory usage: 11.3+ MB

First 5 rows:


,Name,Weight,Position,Market Value,CUSIP,BBID,ISIN,Fund Flows,Company_Name,Industry,Stock_Price,ETF Market Value
Date,,,,,,,,,,,,
2014-10-31,MELI US Equity,0.014650,424.0,57727.60,58733R102,BBG000GQPB11,US58733R1023,0.0,MercadoLibre Inc,Broadline Retail,136.1500,3.940483e+06
2014-10-31,ADSK US Equity,0.020443,1400.0,80556.00,52769106,BBG000BM7HL0,US0527691069,0.0,Autodesk Inc,Software,57.5400,3.940483e+06
2014-10-31,PRLB US Equity,0.026278,1584.0,103546.08,743713109,BBG000BT13B3,US7437131094,0.0,Proto Labs Inc,Machinery,65.3700,3.940483e+06
2014-10-31,WOLF US Equity,0.009714,1216.0,38279.68,977852102,BBG000BG14P4,US9778521024,0.0,Wolfspeed Inc,Semiconductors & Semiconductor Equipment,31.4800,3.940483e+06
2014-10-31,NVDA US Equity,0.030784,248320.0,121304.32,67066G104,BBG000BBJQV0,US67066G1040,0.0,NVIDIA Corp,Semiconductors & Semiconductor Equipment,0.4885,3.940483e+06


In [4]:
# ARK Portfolio Analysis - Small Positions (<1%) Performance Study
# Export all results to Excel without console output

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Step 1: Filter positions with weight < 1%
small_positions = complete_data[complete_data['Weight'] < 0.01].copy()
large_positions = complete_data[complete_data['Weight'] >= 0.01].copy()

# Step 2: Calculate daily returns
def calculate_returns(df):
    df = df.copy().sort_index()
    df['Price_Return'] = df.groupby('Name')['Stock_Price'].pct_change()
    return df

complete_data_with_returns = calculate_returns(complete_data)

# Filter out invalid returns
valid_returns_mask = (
    (complete_data_with_returns['Price_Return'].notna()) & 
    (abs(complete_data_with_returns['Price_Return']) < 2.0)
)

complete_data_clean = complete_data_with_returns[valid_returns_mask]
small_positions = complete_data_clean[complete_data_clean['Weight'] < 0.01].copy()
large_positions = complete_data_clean[complete_data_clean['Weight'] >= 0.01].copy()

small_returns = small_positions['Price_Return']
large_returns = large_positions['Price_Return']

# Step 3: Calculate market value changes
small_positions['Daily_MV_Change'] = small_positions['Market Value'] * small_positions['Price_Return']
large_positions['Daily_MV_Change'] = large_positions['Market Value'] * large_positions['Price_Return']

small_mv_clean = small_positions['Daily_MV_Change'].dropna()
large_mv_clean = large_positions['Daily_MV_Change'].dropna()
small_mv_clean = small_mv_clean[abs(small_mv_clean) < small_mv_clean.quantile(0.999)]
large_mv_clean = large_mv_clean[abs(large_mv_clean) < large_mv_clean.quantile(0.999)]

# Step 4: Calculate Slugging Ratios
small_winning_returns = small_returns[small_returns > 0]
small_losing_returns = small_returns[small_returns < 0]
small_avg_winning_return = small_winning_returns.mean()
small_avg_losing_return = abs(small_losing_returns.mean())
small_slugging_ratio_returns = small_avg_winning_return / small_avg_losing_return if small_avg_losing_return != 0 else np.inf

large_winning_returns = large_returns[large_returns > 0]
large_losing_returns = large_returns[large_returns < 0]
large_avg_winning_return = large_winning_returns.mean()
large_avg_losing_return = abs(large_losing_returns.mean())
large_slugging_ratio_returns = large_avg_winning_return / large_avg_losing_return if large_avg_losing_return != 0 else np.inf

# Win/Loss analysis
small_positive_days = (small_mv_clean > 0).sum()
small_negative_days = (small_mv_clean < 0).sum()
small_win_rate = small_positive_days / len(small_mv_clean)

large_positive_days = (large_mv_clean > 0).sum()
large_negative_days = (large_mv_clean < 0).sum()
large_win_rate = large_positive_days / len(large_mv_clean)

# Extreme values
small_best_day = small_mv_clean.max()
small_worst_day = small_mv_clean.min()
large_best_day = large_mv_clean.max()
large_worst_day = large_mv_clean.min()

small_total_change = small_mv_clean.sum()
large_total_change = large_mv_clean.sum()

small_daily_avg = small_mv_clean.mean()
large_daily_avg = large_mv_clean.mean()

# =============================================================================
# EXPORT TO EXCEL
# =============================================================================

excel_filename = 'ARK_Small_Positions_Analysis.xlsx'
with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
    
    # Sheet 1: Summary Statistics
    summary_data = {
        'Metric': [
            'Total Records',
            'Records with Valid Returns',
            'Mean Daily Return (%)',
            'Median Daily Return (%)',
            'Standard Deviation (%)',
            'Min Daily Return (%)',
            'Max Daily Return (%)',
            '1st Percentile (%)',
            '5th Percentile (%)',
            '25th Percentile (%)',
            '75th Percentile (%)',
            '95th Percentile (%)',
            '99th Percentile (%)',
            'Win Rate (%)',
            'Average Winning Return (%)',
            'Average Losing Return (%)',
            'Slugging Ratio',
            'Total MV Change ($)',
            'Average Daily MV Change ($)',
            'Best Day MV Change ($)',
            'Worst Day MV Change ($)',
            'Risk-Adjusted Return'
        ],
        'Small Positions (<1%)': [
            len(complete_data[complete_data['Weight'] < 0.01]),
            len(small_returns),
            f"{small_returns.mean()*100:.4f}",
            f"{small_returns.median()*100:.4f}",
            f"{small_returns.std()*100:.4f}",
            f"{small_returns.min()*100:.4f}",
            f"{small_returns.max()*100:.4f}",
            f"{np.percentile(small_returns, 1)*100:.4f}",
            f"{np.percentile(small_returns, 5)*100:.4f}",
            f"{np.percentile(small_returns, 25)*100:.4f}",
            f"{np.percentile(small_returns, 75)*100:.4f}",
            f"{np.percentile(small_returns, 95)*100:.4f}",
            f"{np.percentile(small_returns, 99)*100:.4f}",
            f"{small_win_rate*100:.2f}",
            f"{small_avg_winning_return*100:.4f}",
            f"{small_avg_losing_return*100:.4f}",
            f"{small_slugging_ratio_returns:.4f}",
            f"{small_total_change:,.2f}",
            f"{small_daily_avg:,.2f}",
            f"{small_best_day:,.2f}",
            f"{small_worst_day:,.2f}",
            f"{(small_returns.mean()/small_returns.std()):.4f}"
        ],
        'Large Positions (≥1%)': [
            len(complete_data[complete_data['Weight'] >= 0.01]),
            len(large_returns),
            f"{large_returns.mean()*100:.4f}",
            f"{large_returns.median()*100:.4f}",
            f"{large_returns.std()*100:.4f}",
            f"{large_returns.min()*100:.4f}",
            f"{large_returns.max()*100:.4f}",
            f"{np.percentile(large_returns, 1)*100:.4f}",
            f"{np.percentile(large_returns, 5)*100:.4f}",
            f"{np.percentile(large_returns, 25)*100:.4f}",
            f"{np.percentile(large_returns, 75)*100:.4f}",
            f"{np.percentile(large_returns, 95)*100:.4f}",
            f"{np.percentile(large_returns, 99)*100:.4f}",
            f"{large_win_rate*100:.2f}",
            f"{large_avg_winning_return*100:.4f}",
            f"{large_avg_losing_return*100:.4f}",
            f"{large_slugging_ratio_returns:.4f}",
            f"{large_total_change:,.2f}",
            f"{large_daily_avg:,.2f}",
            f"{large_best_day:,.2f}",
            f"{large_worst_day:,.2f}",
            f"{(large_returns.mean()/large_returns.std()):.4f}"
        ],
        'Difference (Small - Large)': [
            len(complete_data[complete_data['Weight'] < 0.01]) - len(complete_data[complete_data['Weight'] >= 0.01]),
            len(small_returns) - len(large_returns),
            f"{(small_returns.mean() - large_returns.mean())*100:.4f}",
            f"{(small_returns.median() - large_returns.median())*100:.4f}",
            f"{(small_returns.std() - large_returns.std())*100:.4f}",
            f"{(small_returns.min() - large_returns.min())*100:.4f}",
            f"{(small_returns.max() - large_returns.max())*100:.4f}",
            f"{(np.percentile(small_returns, 1) - np.percentile(large_returns, 1))*100:.4f}",
            f"{(np.percentile(small_returns, 5) - np.percentile(large_returns, 5))*100:.4f}",
            f"{(np.percentile(small_returns, 25) - np.percentile(large_returns, 25))*100:.4f}",
            f"{(np.percentile(small_returns, 75) - np.percentile(large_returns, 75))*100:.4f}",
            f"{(np.percentile(small_returns, 95) - np.percentile(large_returns, 95))*100:.4f}",
            f"{(np.percentile(small_returns, 99) - np.percentile(large_returns, 99))*100:.4f}",
            f"{(small_win_rate - large_win_rate)*100:.2f}",
            f"{(small_avg_winning_return - large_avg_winning_return)*100:.4f}",
            f"{(small_avg_losing_return - large_avg_losing_return)*100:.4f}",
            f"{small_slugging_ratio_returns - large_slugging_ratio_returns:.4f}",
            f"{small_total_change - large_total_change:,.2f}",
            f"{small_daily_avg - large_daily_avg:,.2f}",
            f"{small_best_day - large_best_day:,.2f}",
            f"{small_worst_day - large_worst_day:,.2f}",
            f"{(small_returns.mean()/small_returns.std()) - (large_returns.mean()/large_returns.std()):.4f}"
        ],
        'Winner': [
            'Small' if len(complete_data[complete_data['Weight'] < 0.01]) > len(complete_data[complete_data['Weight'] >= 0.01]) else 'Large',
            'Small' if len(small_returns) > len(large_returns) else 'Large',
            'Small' if small_returns.mean() > large_returns.mean() else 'Large',
            'Small' if small_returns.median() > large_returns.median() else 'Large',
            'Small' if small_returns.std() < large_returns.std() else 'Large',
            'Small' if small_returns.min() > large_returns.min() else 'Large',
            'Small' if small_returns.max() > large_returns.max() else 'Large',
            'Small' if np.percentile(small_returns, 1) > np.percentile(large_returns, 1) else 'Large',
            'Small' if np.percentile(small_returns, 5) > np.percentile(large_returns, 5) else 'Large',
            'Small' if np.percentile(small_returns, 25) > np.percentile(large_returns, 25) else 'Large',
            'Small' if np.percentile(small_returns, 75) > np.percentile(large_returns, 75) else 'Large',
            'Small' if np.percentile(small_returns, 95) > np.percentile(large_returns, 95) else 'Large',
            'Small' if np.percentile(small_returns, 99) > np.percentile(large_returns, 99) else 'Large',
            'Small' if small_win_rate > large_win_rate else 'Large',
            'Small' if small_avg_winning_return > large_avg_winning_return else 'Large',
            'Small' if small_avg_losing_return < large_avg_losing_return else 'Large',
            'Small' if small_slugging_ratio_returns > large_slugging_ratio_returns else 'Large',
            'Small' if small_total_change > large_total_change else 'Large',
            'Small' if small_daily_avg > large_daily_avg else 'Large',
            'Small' if small_best_day > large_best_day else 'Large',
            'Small' if small_worst_day > large_worst_day else 'Large',
            'Small' if (small_returns.mean()/small_returns.std()) > (large_returns.mean()/large_returns.std()) else 'Large'
        ]
    }
    
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_excel(writer, sheet_name='Summary_Analysis', index=False)
    
    # Sheet 2: Key Insights
    insights_data = {
        'Performance Category': [
            'Overall Daily Performance',
            'Risk Management (Lower Volatility)', 
            'Consistency (Higher Win Rate)',
            'Winner Quality (Slugging Ratio)',
            'Extreme Day Performance',
            'Market Value Generation',
            'Risk-Adjusted Returns',
            'Final Verdict'
        ],
        'Winner': [
            'Small Positions' if small_returns.mean() > large_returns.mean() else 'Large Positions',
            'Small Positions' if small_returns.std() < large_returns.std() else 'Large Positions',
            'Small Positions' if small_win_rate > large_win_rate else 'Large Positions',
            'Small Positions' if small_slugging_ratio_returns > large_slugging_ratio_returns else 'Large Positions',
            'Small Positions' if small_returns.max() > large_returns.max() else 'Large Positions',
            'Small Positions' if small_total_change > large_total_change else 'Large Positions',
            'Small Positions' if (small_returns.mean()/small_returns.std()) > (large_returns.mean()/large_returns.std()) else 'Large Positions',
            'Small Positions' if sum([
                small_returns.mean() > large_returns.mean(),
                small_returns.std() < large_returns.std(),
                small_win_rate > large_win_rate,
                small_slugging_ratio_returns > large_slugging_ratio_returns,
                small_total_change > large_total_change,
                (small_returns.mean()/small_returns.std()) > (large_returns.mean()/large_returns.std())
            ]) >= 4 else 'Large Positions'
        ],
        'Key Insight': [
            f"Daily edge of {abs(small_returns.mean() - large_returns.mean())*100:.3f}% per day",
            f"Daily volatility {'advantage' if small_returns.std() < large_returns.std() else 'disadvantage'} of {abs(small_returns.std() - large_returns.std())*100:.2f}%",
            f"Win {abs(small_win_rate - large_win_rate)*100:.1f}% more trading days",
            f"Winners generate {max(small_slugging_ratio_returns, large_slugging_ratio_returns):.2f}x vs losers daily",
            f"Best day advantage: {abs(small_returns.max() - large_returns.max())*100:.2f}%",
            f"Total dollar advantage: ${abs(small_total_change - large_total_change):,.0f}",
            f"Better risk-return by {abs((small_returns.mean()/small_returns.std()) - (large_returns.mean()/large_returns.std())):.3f}",
            f"Dominate {sum([small_returns.mean() > large_returns.mean(), small_returns.std() < large_returns.std(), small_win_rate > large_win_rate, small_slugging_ratio_returns > large_slugging_ratio_returns, small_total_change > large_total_change, (small_returns.mean()/small_returns.std()) > (large_returns.mean()/large_returns.std())])}/6 key metrics"
        ]
    }
    
    insights_df = pd.DataFrame(insights_data)
    insights_df.to_excel(writer, sheet_name='Key_Insights', index=False)

# Auto-adjust column widths
from openpyxl import load_workbook

wb = load_workbook(excel_filename)
for sheet_name in wb.sheetnames:
    ws = wb[sheet_name]
    for column in ws.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                max_length = max(max_length, len(str(cell.value)))
            except:
                pass
        ws.column_dimensions[column_letter].width = min(max_length + 2, 50)

wb.save(excel_filename)

print(f"✅ ARK Small Positions Analysis exported to '{excel_filename}'")

✅ ARK Small Positions Analysis exported to 'ARK_Small_Positions_Analysis.xlsx'


In [5]:
# Export small positions stock analysis
import pandas as pd
import numpy as np

# Filter stocks that were ever < 1% weight
small_positions_data = complete_data_clean[complete_data_clean['Weight'] < 0.01].copy()

# Calculate daily MV change
small_positions_data['Daily_MV_Change'] = small_positions_data['Market Value'] * small_positions_data['Price_Return']

# Get unique tickers that were ever small positions
small_position_tickers = small_positions_data['Name'].unique()

# Calculate metrics for each stock
stock_results = []

for ticker in small_position_tickers:
    # Get data for this ticker when weight < 1%
    ticker_data = small_positions_data[small_positions_data['Name'] == ticker].copy()
    
    # Get company name
    company_name = ticker_data['Company_Name'].dropna().iloc[0] if len(ticker_data['Company_Name'].dropna()) > 0 else 'N/A'
    
    # Calculate metrics
    returns = ticker_data['Price_Return'].dropna()
    mv_changes = ticker_data['Daily_MV_Change'].dropna()
    
    if len(returns) == 0:
        continue
    
    mean_return = returns.mean()
    std_return = returns.std()
    total_mv_change = mv_changes.sum()
    
    # Calculate slugging ratio
    winning_returns = returns[returns > 0]
    losing_returns = returns[returns < 0]
    
    if len(winning_returns) > 0 and len(losing_returns) > 0:
        avg_winning_return = winning_returns.mean()
        avg_losing_return = abs(losing_returns.mean())
        slugging_ratio = avg_winning_return / avg_losing_return if avg_losing_return != 0 else np.inf
    else:
        slugging_ratio = np.nan
    
    stock_results.append({
        'Ticker': ticker,
        'Company Name': company_name,
        'Total MV Change ($)': total_mv_change,
        'Slugging Ratio': slugging_ratio,
        'Mean Daily Return (%)': mean_return * 100,
        'Standard Deviation (%)': std_return * 100
    })

# Create dataframe
results_df = pd.DataFrame(stock_results)

# Export to excel
excel_filename = 'Small_Positions_Stock_Analysis.xlsx'
results_df.to_excel(excel_filename, index=False)

print(f"Exported to {excel_filename}")

Exported to Small_Positions_Stock_Analysis.xlsx
